In [3]:
import cv2
import mediapipe as mp
import numpy as np

In [4]:
def calc_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [ ]:
def breaking_legs(strokes, arm_angle, leg_angle):
    # var = False
    # if after finish
     # if leg_angle > 170 -ish 
       # if catch angle < --- 
            # var = true 
    
    #return bool?
    pass 
    

In [68]:
def display():
    # call breaking legs  # maybe also add other metertics about at what angle
    # get adv layback from list given
    # somthing else
    pass


SyntaxError: expected ':' (428108329.py, line 1)

In [5]:
cam = cv2.VideoCapture('rowing1.MOV')

drawing = mp.solutions.drawing_utils
body = mp.solutions.pose  

frame_rate = cam.get(cv2.CAP_PROP_FPS)
elapsed_time = 0.0 
stroke_t = 0.0

with body.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    strokes = 0
    stage = ''
    layback = 0
    time = 0
    while cam.isOpened():
        ret, frame = cam.read()
        if not ret:
            break
        elapsed_time += 1.0 / frame_rate
        
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False

        results = pose.process(img)
        
        img.flags.writeable = True 
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        #previous_wrist = 0
        positive = True
        try:
            landmarks = results.pose_landmarks.landmark
            
            L_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]

            L_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]

            L_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]

            L_ankle = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].x,
                    landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].y]

            L_wrist = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x]

           
            
            
            # if positive: 
            #     if previous_wrist - L_wrist[0] > 0:
            #         # mark
            #         print("stroke une")
            #         stroke_time = 0.0
            #         positive = False
                    
            # else:
            #     if previous_wrist - L_wrist[0] < 0:
            #         positive = True
                                    
            # previous_wrist = L_wrist[0]
            
           
            
            
            
            angle = calc_angle(L_shoulder, L_hip, L_knee)
            angle2 = calc_angle(L_ankle, L_knee, L_hip)
            
            cv2.putText(img, str(round(angle, 2)),          
                           tuple(np.multiply(L_shoulder, [1800, 1000]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA
                                )

        
            cv2.putText(img, str(round(angle2, 2)),       
                           tuple(np.multiply(L_knee, [1800, 1000]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA
                                )  
            
            # what to here 

            
            
            
        except:
            pass
        
        
        
        
        
        if angle > 100:
            # if layback < angle:
            #     layback = angle
            # # if strokes > 1: add to display
           stage = "fin"
        if angle < 60 and stage =='fin':
            # add layback to adv list to crucnch maybe a list and set layback back to angle
            # store the angeles here and well as
            # angle of breaking knees 
            # ^ for the bool as well as the angle it slef adv
            
            if strokes > 0:
                time = int(60/(elapsed_time - stroke_t))
            stroke_t = elapsed_time
            stage="catch"
            strokes +=1
          
        
        
        
        cv2.rectangle(img, (0,0), (750,80), (240,46,56), -1)
        cv2.rectangle(img, (0,0), (125,250), (240,46,56), -1)
        
        # strokes
        cv2.putText(img, 'Strokes', (14,24), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(img, str(strokes), 
                    (20,75), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        # total time
        cv2.putText(img, f'Elapsed Time: {elapsed_time:.2f} seconds', (150, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 0), 2, cv2.LINE_AA)
            #stroke rate
        cv2.putText(img, 'S/m', (20,150), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
            
        cv2.putText(img, str(time), (20,200),         
                     cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA
                                )  

        
        if results.pose_landmarks:
            drawing.draw_landmarks(img, results.pose_landmarks, body.POSE_CONNECTIONS,
                                   drawing.DrawingSpec(color=(255, 255, 255), thickness=2, circle_radius=2),
                                   drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                                  )
        
        
        
        
        cv2.namedWindow('footie_training', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('footie_training', 800, 600)
        cv2.imshow("footie_training", img)
        
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break
            display()
        
         
cam.release()
cv2.destroyAllWindows()




In [26]:
L_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
              landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]

L_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
         landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]

L_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x,
          landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]

L_heel = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HEEL.value].x,
          landmarks[mp.solutions.pose.PoseLandmark.LEFT_HEEL.value].y]

print("Left Shoulder:", L_shoulder)
print("Left Hip:", L_hip)
print("Left Heel:", L_heel)

Left Shoulder: [0.424895703792572, 0.20157688856124878]
Left Hip: [0.47357648611068726, 0.5483577847480774]
Left Heel: [0.32899168133735657, 0.7550986409187317]


In [15]:
calc_angle(L_shoulder, L_hip, L_heel)

132.80453514226213